In [25]:
import os
import torch

import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.utils import save_image
from torch.utils.data import DataLoader

In [26]:
batch_size = 128
epochs = 10
fig_folder = "results"

os.makedirs(fig_folder, exist_ok=True)

In [27]:
train_loader = DataLoader(
    datasets.MNIST('data', train=True, download=True,
                   transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=True)
test_loader = DataLoader(
    datasets.MNIST('data', train=False, transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=False)

In [28]:
class VAE(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(784, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)

    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc21(h1), self.fc22(h1)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 784))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

# Reconstruction + KL divergence losses summed over all elements and batch
def vae_loss(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD

In [29]:
def train(model, loss_function, epoch, log_interval=100):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))

    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))


def test(model, loss_function, epoch, batch_size, fig_folder):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()
            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n],
                                      recon_batch.view(batch_size, 1, 28, 28)[:n]])
                save_image(comparison.cpu(),
                         f'{fig_folder}/reconstruction_{epoch}.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))


In [30]:
vae_model = VAE()

optimizer = optim.Adam(vae_model.parameters(), lr=1e-3)


for epoch in range(1, epochs + 1):
    train(model=vae_model, loss_function=vae_loss, epoch=epoch)
    test(model=vae_model, loss_function=vae_loss, epoch=epoch, batch_size=batch_size,
         fig_folder=fig_folder)
    with torch.no_grad():
        sample = torch.randn(64, 20)
        sample = vae_model.decode(sample).cpu()
        save_image(sample.view(64, 1, 28, 28),
                   'results/sample_' + str(epoch) + '.png')

Train Epoch: 1 [0/60000 (0%)]	Loss: 549.312073
Train Epoch: 1 [12800/60000 (21%)]	Loss: 186.265244
Train Epoch: 1 [25600/60000 (43%)]	Loss: 153.820740
Train Epoch: 1 [38400/60000 (64%)]	Loss: 142.583405
Train Epoch: 1 [51200/60000 (85%)]	Loss: 132.914871
====> Epoch: 1 Average loss: 165.7666
====> Test set loss: 128.6223
Train Epoch: 2 [0/60000 (0%)]	Loss: 128.845352
Train Epoch: 2 [12800/60000 (21%)]	Loss: 117.934227
Train Epoch: 2 [25600/60000 (43%)]	Loss: 119.007568
Train Epoch: 2 [38400/60000 (64%)]	Loss: 115.397507
Train Epoch: 2 [51200/60000 (85%)]	Loss: 120.260277
====> Epoch: 2 Average loss: 122.0266
====> Test set loss: 116.0449
Train Epoch: 3 [0/60000 (0%)]	Loss: 119.533386
Train Epoch: 3 [12800/60000 (21%)]	Loss: 112.502136
Train Epoch: 3 [25600/60000 (43%)]	Loss: 112.276855
Train Epoch: 3 [38400/60000 (64%)]	Loss: 115.068977
Train Epoch: 3 [51200/60000 (85%)]	Loss: 109.649689
====> Epoch: 3 Average loss: 114.8643
====> Test set loss: 111.9633
Train Epoch: 4 [0/60000 (0%)]	L